In [ ]:
# Trubo state is not a real package, but a method to find a trust region to help exploit a complex function. You can follow the tutorial on botorch to learn how to do this on a 20D Ackely function. But I'll leave this here for now

from botorch.models.turbo import TurboState
from botorch.optim import optimize_acqf
from botorch.acquisition import qExpectedImprovement
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood


noise_level = 0 

c:\Users\uqkmuroi\gitcode\Torch-Practice\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'botorch.models.turbo'

In [1]:
def objective_function(X, pHopt =7, pHopt2 = 5.5, temp_opt =35, temp_opt2=30,  a = 100, b = 20, c = 1, noise_level = noise_level, seed = None):
    """
    Simulates a 2D Gaussian-like response surface with controllable noise.

    Parameters:
    - X: input tensor of shape [n, 2], columns are [pH, temp]
    - pHopt, temp_opt: optimal pH and temperature
    - a: peak value (must be high enough to keep output positive)
    - b, c: curvature coefficients (bigger = narrower peak)
    - noise_level: fraction of y to scale the noise (e.g., 0.1 = 10%)

    Returns:
    - y: simulated noisy response values
    """
    if seed is not None:
        torch.manual_seed(seed) # set for reproducibility
    pH, temp = X[:, 0], X[:, 1]

     # First peak at (pH=7, temp=35)
    pH_term1 = torch.exp(-0.5 * ((pH - pHopt) / 1.5)**2)  # Gaussian term for pH with width 1.5
    temp_term1 = torch.exp(-0.5 * ((temp - temp_opt) / 5.0)**2)  # Gaussian term for temp with width 5.0

    # Second peak at (pH=5.5, temp=30)
    pH_term2 = torch.exp(-0.5 * ((pH - pHopt2) / 1.5)**2)
    temp_term2 = torch.exp(-0.5 * ((temp - temp_opt2) / 5.0)**2)

     # Stronger Sinusoidal Modulation
    sin_component = torch.sin(2 * pH) * torch.cos(1.5 * temp)  # Higher frequency
    wave_strength = 1.5  # Scale up the wave effect

    # Combine the two peaks with the stronger sinusoidal variation
    y = (pH_term1 * temp_term1 + pH_term2 * temp_term2) * (1 + wave_strength * sin_component)

    noise = noise_level * torch.randn_like(y) # changed this here from Isabella's to make noise homoscedastic (same for all points)
    return y + noise
'''
# generates initial sample using latinhypercube
g=torch.Generator().manual_seed(42) # ensure's results are same each time
train_X = maximin_lhs(initial_sample, bounds, generator=g) # this will be consistent as we set the seed at beginning
'''
# above is broken, below is temp fix
train_X = scaled_lhs_design
# generates solution for initial sample using objective function
train_Y= objective_function(train_X, seed=seed) # add seed=seed here if you want obj func to always spit out the same answer. Removing it will simulate biological variability. Usually not necessary for initial run unless samples repeated.
train_Y = train_Y.unsqueeze(-1) # reshape to (n, 1) (from (n,))

NameError: name 'noise_level' is not defined

In [ ]:
# Turbo MODEL


# Fit initial GP
gp = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_model(mll)

# Set up TuRBO state
turbo_state = TurboState(
    dim=train_X.shape[-1],
    batch_size=1,
    length_init=0.8,  # large init trust region
    success_tolerance=3,
    failure_tolerance=3,
)

# Main loop
for iteration in range(MAX_ITERS):
    if turbo_state.restart_triggered:
        turbo_state = TurboState(dim=train_X.shape[-1], batch_size=1)

    # Define acquisition function
    acq = qExpectedImprovement(model=gp, best_f=train_Y.max(), maximize=True)

    # Optimize in trust region
    candidates, _ = optimize_acqf(
        acq_function=acq,
        bounds=turbo_state.get_bounds(train_X),
        q=1,
        num_restarts=5,
        raw_samples=100,
    )

    # Evaluate candidate (real experiment)
    new_Y = objective_function(candidates)

    # Update data
    train_X = torch.cat([train_X, candidates], dim=0)
    train_Y = torch.cat([train_Y, new_Y], dim=0)

    # Update model
    gp.set_train_data(train_X, train_Y, strict=False)
    fit_gpytorch_model(ExactMarginalLogLikelihood(gp.likelihood, gp))

    # Update TuRBO trust region
    turbo_state.update(train_Y[-1])


SyntaxError: invalid syntax (2431776247.py, line 11)